## 1. Import Modules

In [0]:
!pip install xgboost

In [0]:
import pandas as pd
import plotly.express as px
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GroupShuffleSplit
import xgboost as xgb
from sklearn.metrics import ndcg_score

## 2. Read Files

In [0]:
bizDF = (spark.read
  .option("inferSchema", True)
  .json("/FileStore/tables/Project_Data_Extract/business_filtered.json"))

bizDF.display()

address,business_id,business_perf,categories,city,cluster,latitude,longitude,name,postal_code,review_ave_stars,review_count,review_ss,state,tips_count,tips_ss
3164 NW 185th Ave,Agq4zoNLSIpT1_ZJbnrvww,Good,"Food, Donuts",portland,1,45.5429252,-122.8662879,Donut Palace,97229,4.285714285714286,173,0.8146031746031747,OR,13,0.4414153846153846
187 Elm St,bxy3khT-2R66tcdKjFa2pw,Good,"Restaurants, Seafood, Italian, Pizza",somerville,1,42.3930843,-71.1202673,Posto,02144,4.1866666666666665,910,0.8298826666666667,MA,90,0.34740777777777776
"12251 Number 1 Road, Suite 140",3KqpiLDAjeeMmZeU-Il_ng,Poor,"Japanese, Restaurants, Sushi Bars",richmond,2,49.1245685,-123.1815968,Yokohama Teppanyaki,V7E 1T6,2.0833333333333335,80,0.27915833333333334,BC,8,0.3193
1487 Dorchester Ave,TYDCjEMga3cm7k638hKhTg,Good,"Chinese, Vietnamese, Restaurants",dorchester,1,42.2999809,-71.0604396,Hien Vuong Restaurant,02122,4.142857142857143,36,0.7781142857142858,MA,9,0.2989666666666667
6815 E Broad St,KpdYtK2KtLxEzYNW38XKZQ,Poor,"Restaurants, Fast Food, Mexican",columbus,2,39.9806544587,-82.821739,Chipotle Mexican Grill,43213,1.7857142857142858,43,0.09112857142857145,OH,12,-0.014208333333333331
"730 W Sand Lake Rd, Ste 30",f3teByaeIKPTYetAawUtxg,Good,"Restaurants, Breakfast & Brunch, Cuban, Latin American",orlando,1,28.4500355797,-81.3879469963,Café Pinar,32809,4.580645161290323,167,0.7474806451612903,FL,25,0.528368
1410 State Rd 436,GrLNk_EE831QDAhaSAZpWA,Good,"Latin American, Cocktail Bars, Restaurants, Colombian, Gastropubs, Nightlife, Bars",casselberry,1,28.627566,-81.316263,Capachos Bar & Grill,32707,3.888888888888889,17,0.8434055555555555,FL,4,0.316575
4315 W Lake Mary Blvd,Hhc8Ix8sygBjqFbiO1c1TQ,Average,"Mexican, Fast Food, Restaurants",lake mary,0,28.7556732126,-81.350758338,Chipotle Mexican Grill,32746,2.75,60,0.29228750000000003,FL,28,0.35261785714285715
197 Massachusetts Ave,zeqEqgEmTY3c7HkJmHrw2g,Average,"Italian, Sandwiches, Fast Food, Pizza, Restaurants",lexington,0,42.4261588,-71.196419,Nick's Place,02420,3.1666666666666665,89,0.5297499999999999,MA,10,0.46006
"520 SW 6th Ave, Ste 105",O_BAT_rvszHYBNEM6z0tcQ,Poor,"Sandwiches, Desserts, Food, Salad, Fast Food, Restaurants",portland,2,45.5199352739,-122.677917119,Subway,97204,2.0,20,-0.3145200000000001,OR,6,0.33155


In [0]:
reviewsDF = (spark.read
  .option("inferSchema", True)
  .json("/FileStore/tables/Project_Data_Extract/reviews.json"))

reviewsDF.display()


aspect_category,business_id,comp_score,sentiment,stars,text
SERVICE#GENERAL,SfkRN960vVQ74mbowSEuXQ,0.8074,positive,5.0,affordable price fast service friendly dog name rambo place great
FOOD#QUALITY,YZs1gNSh_sN8JmN_nrpxeA,0.8805,positive,5.0,one favorite bbq place town meat side fantastic smell like table locate inside smoker noise level manageable highly recommend
FOOD#QUALITY,gGvNgShksetPoimyKV8I9Q,-0.6486,negative,1.0,rate food walk seat wait minute waitress approach state would back take drink order come back another minute later take drink order wait minute get even drink decide leave bad service I ever even busy
FOOD#QUALITY,ttZRucfe1fcN_DX2ZGZ1Vg,0.9775,positive,3.0,restaurant conveniently locate street fox easy spot grab quick bite show sunday although restaurant italian fairly limited selection pasta service inconsistent despite extremely crowd flag waitress time refill bread monday friday offer wine despite go sunday bartender gracious enough honor special friend order gorgonzola salad chicken take well min deliver table salad average bit overprice may due area bruschetta flavorful calamari well season restaurant good spot partake wine week service leaf little desire however hopefully would well week
SERVICE#GENERAL,R0IJhEI-zSJpYT1YN_orWQ,0.9607,positive,5.0,nail salon amaze nikki thao kathy anna talented really listen customer salon amaze amount color I see area also stay trend fun design love place
FOOD#QUALITY,qDyZQ6mT1JajQ7_Yj7PeLw,0.994,positive,4.0,food star service star I go average star really great spot especially shopping domain want drop nonpeak time since get pretty busy love option sit bar get small party first come first serve goto spot crave japanese beer tap favorite spicy chicken ramen get spiciest level good heat nice veggie umami flavor thin noodle number appetizer well standouts brussel sprout crispy chicken also get great combo add thing like crispy chicken salad ramen order fiance often split food go beer real complaint service leaf something desire well often ask thing twice go hunt waiter ask water napkin etc order drink get wait minute food also friendliest staff experienced though get feel lot time feel slammedoverwhelmed high volume customer still pleasant experience crowd restaurant think unreasonable want friendliermore helpful staff overall food beerhot sake tap make worthwhile go try go happy houroff time
SERVICE#GENERAL,X7ZPCV8jyS5rw_bI0aWNNA,0.9468,positive,5.0,rent local spot large family reunion conveniently locate amaze service friendliest reception meaningful reunion definitely book thank make family feel like royalty
FOOD#QUALITY,3Ck6jp7qFRB4twIOXWA2Pw,-0.4653,negative,2.0,reason give dillards two star instead one quality merchandise customer service get negative last time perfume counter look sample five minute saw sale lady help three people come one couple even interested perfume stand near counter still come way help ready buy holiday gift felt disrespect go another sale lady ask elevator even look register point without word corner store even elevator locate also pretend see show interest jacket department left store without give business many quality store quality merchandise quality service tolerate arrogance disrespect prejudice anyone go shopping positive experience negative one
SERVICE#GENERAL,Y2cj0TQvfSdRU3H55CkEgA,-0.7783,negative,1.0,service awful charge wrong size inform worker told even matter
FOOD#QUALITY,XxixtrEXLI9Rr1Ojaq70Zw,0.8625,positive,5.0,casual atmosphere great view service food delicious drink hit spot


In [0]:
tipsDF = (spark.read
  .option("inferSchema", True)
  .json("/FileStore/tables/Project_Data_Extract/dataset_tips.json"))

tipsDF.display()

business_id,date,text_w/o_stopwords,tips_comp_score
ENwBByjpoa5Gg7tKgxqwLg,2011-07-22T19:07:35.000Z,carne asada chips,0.0
jKO4Og6ucdX2-YCTKQVYjg,2014-09-10T07:33:29.000Z,best happy hour pm pm martinis fridays nothing pink delicious,0.7097
9Bto7mky640ocgezVKSfVg,2013-12-13T23:23:41.000Z,nice people skilled staff clean location think I ever taken time years,0.6705
XWFjKtRGZ9khRGtGg2ZvaA,2017-07-11T23:07:16.000Z,price bowling old fashion excellent drink help bowling score,0.7506
mkrx0VhSMU3p3uhyJGCoWA,2016-11-30T08:46:36.000Z,solid golds great sauna great staff even two,0.8689
VQftVUvHfMQdDTmnO0iQqg,2010-03-22T21:40:26.000Z,try different styles instructors discover level classes moving higher levels,0.0
2PxZ-fICnd432NJHefXrcA,2011-10-08T17:14:20.000Z,border fresh fast nonfast food chain minitacos perfect fresh avocado black fabulous,0.8934
oQyf1788YWsiDLupGva6sw,2015-06-09T14:35:57.000Z,cold cuts best,0.4588
OQ2oHkcWA8KNC1Lsvj1SBA,2018-03-22T15:28:09.000Z,water ave coffee served l jeans,0.0
Wqetc51pFQzz04SXh_AORA,2014-06-07T12:09:55.000Z,busy,0.0


In [0]:
tips_pdf = tipsDF.toPandas()
reviews_pdf = reviewsDF.toPandas()
biz_pdf = bizDF.toPandas()

## 3. Calculate Satsifaction Score (Jia Ying)
## Sentiment Score was predicted by Shi Yao

In [0]:
reviews_pdf['happy_customers']=0
for i in range(len(reviews_pdf)):
    if reviews_pdf['sentiment'][i]=="positive" and float(reviews_pdf['stars'][i])>3:
        reviews_pdf['happy_customers'][i]=1
    else:
        reviews_pdf['happy_customers'][i]=0

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<command-1781364522751924> in <module>
      1 reviews_pdf['happy_customers']=0
      2 for i in range(len(reviews_pdf)):
----> 3     if reviews_pdf['sentiment'][i]=="positive" and float(reviews_pdf['stars'][i])>3 and tips_pdf["tips_comp_score"]>=0.05:
      4         reviews_pdf['happy_customers'][i]=1
      5     else:

/databricks/python/lib/python3.8/site-packages/pandas/core/generic.py in __nonzero__(self)
   1440     @final
   1441     def __nonzero__(self):
-> 1442         raise ValueError(
   1443             f"The truth value of a {type(self).__name__} is ambiguous. "
   1444             "Use a.empty, a.bool(), a.item(), a.any() or a.all()."

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [0]:
biz_id_df=pd.DataFrame(reviews_pdf['business_id'].value_counts())
biz_id_df.rename(columns={"business_id":"count"},inplace=True)
biz_id_df['business_id']=biz_id_df.index
biz_id_df.reset_index(inplace=True)
biz_id_df.drop(columns=['index'],inplace=True)
biz_id_df

,count,business_id
0,1797,YZs1gNSh_sN8JmN_nrpxeA
1,1734,GJxFtnTqTiokFedNrW9iDQ
2,1156,OQ2oHkcWA8KNC1Lsvj1SBA
3,1031,viqkunBBI6QOIZehME-7gg
4,973,cs6HfZNykLVitm09jWFqWg
...,...,...
116626,1,z8PEBckibjrqjkQBbQPaSA
116627,1,RP3LQCq501T2nmef6Xmy4g
116628,1,hM3-_1Ku3gt0a5vOtK8mbw
116629,1,-fXCgMV289aUljFUOgXz4w


In [0]:
biz_id_df=pd.DataFrame(reviews_pdf['business_id'].value_counts())
biz_id_df.rename(columns={"business_id":"count"},inplace=True)
biz_id_df['business_id']=biz_id_df.index
biz_id_df.reset_index(inplace=True)
biz_id_df.drop(columns=['index'],inplace=True)
biz_id_df

,count,business_id
0,1797,YZs1gNSh_sN8JmN_nrpxeA
1,1734,GJxFtnTqTiokFedNrW9iDQ
2,1156,OQ2oHkcWA8KNC1Lsvj1SBA
3,1031,viqkunBBI6QOIZehME-7gg
4,973,cs6HfZNykLVitm09jWFqWg
...,...,...
116626,1,z8PEBckibjrqjkQBbQPaSA
116627,1,RP3LQCq501T2nmef6Xmy4g
116628,1,hM3-_1Ku3gt0a5vOtK8mbw
116629,1,-fXCgMV289aUljFUOgXz4w


In [0]:
happy_cust_df=pd.DataFrame(reviews_pdf.groupby('business_id').sum()['happy_customers'])
happy_cust_df['business_id']=happy_cust_df.index
happy_cust_df.reset_index(drop=True,inplace=True)
happy_cust_df=happy_cust_df[["happy_customers","business_id"]]
#happy_cust_df1=happy_cust_df[['happy_customers',"business_id']]
happy_cust_df

,happy_customers,business_id
0,0,--0DF12EMHYI8XIgoFha6A
1,2,--0r8K_AQ4FZfLsX3ZYRDA
2,0,--2aF9NhXnNVpDV0KS3xBQ
3,0,--2mEJ63SC_8_08_jGgVIg
4,1,--4INAzazK6omgf3mY3dfw
...,...,...
116626,0,zzkku5rAHjABjQqBDiGo7w
116627,8,zzlkjDG9Rv8Jn-vSolMgyw
116628,1,zzngBb9MoGpkFe8tfowvvA
116629,0,zzpmoTVq4yn86U7ArHyFBQ


In [0]:
CSAT_df=biz_id_df.merge(happy_cust_df,how='left',on="business_id")
CSAT_df

,count,business_id,happy_customers
0,1797,YZs1gNSh_sN8JmN_nrpxeA,399
1,1734,GJxFtnTqTiokFedNrW9iDQ,450
2,1156,OQ2oHkcWA8KNC1Lsvj1SBA,354
3,1031,viqkunBBI6QOIZehME-7gg,302
4,973,cs6HfZNykLVitm09jWFqWg,273
...,...,...,...
116626,1,z8PEBckibjrqjkQBbQPaSA,1
116627,1,RP3LQCq501T2nmef6Xmy4g,0
116628,1,hM3-_1Ku3gt0a5vOtK8mbw,0
116629,1,-fXCgMV289aUljFUOgXz4w,0


In [0]:
CSAT_df['Cust_Sat_Score']=round((CSAT_df['happy_customers']/CSAT_df['count']),2)

In [0]:
df = px.data.tips()
fig = px.box(CSAT_df, y="Cust_Sat_Score")
fig.show()

In [0]:
CSAT_df['Cust_Sat_Score']=CSAT_df['Cust_Sat_Score']*100
# CSAT_df['Cust_Sat_Score']=CSAT_df['Cust_Sat_Score'].astype('str')
# CSAT_df['Cust_Sat_Score']=CSAT_df['Cust_Sat_Score']+'%'

In [0]:
CSAT_df

,count,business_id,happy_customers,Cust_Sat_Score
0,1797,YZs1gNSh_sN8JmN_nrpxeA,399,22.0
1,1734,GJxFtnTqTiokFedNrW9iDQ,450,26.0
2,1156,OQ2oHkcWA8KNC1Lsvj1SBA,354,31.0
3,1031,viqkunBBI6QOIZehME-7gg,302,29.0
4,973,cs6HfZNykLVitm09jWFqWg,273,28.0
...,...,...,...,...
116626,1,z8PEBckibjrqjkQBbQPaSA,1,100.0
116627,1,RP3LQCq501T2nmef6Xmy4g,0,0.0
116628,1,hM3-_1Ku3gt0a5vOtK8mbw,0,0.0
116629,1,-fXCgMV289aUljFUOgXz4w,0,0.0


## 4. Rank Businesses by State (Varsha)

In [0]:
biz_rank_pdf = biz_pdf.merge(CSAT_df[["business_id","Cust_Sat_Score"]])

### 4.1 Assign Relevance Score

In [0]:
states=["MA","FL","OR","GA","BC","TX","OH","WA","CO","KS"]

In [0]:
appended_data = []
for i in range(len(states)):
    df_iloc=biz_rank_pdf.loc[biz_rank_pdf['state'] == str(states[i]) ]
    df_iloc["Rank"] = df_iloc[["Cust_Sat_Score", "review_ss","tips_ss"]].apply(tuple, axis = 1).rank(method = 'dense', ascending = False).astype(int)
    appended_data.append(df_iloc)

biz_rank_pdf = pd.concat(appended_data)

<command-1781364522751938>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
state_MA=biz_rank_pdf.loc[biz_rank_pdf['state'] == "MA"]   
bins = np.arange(0, 6220, 20)
labels = np.arange(310, 0, -1)
state_MA['Relevance'] = pd.cut(biz_rank_pdf['Rank'], bins=bins, labels=labels)

<command-1781364522751942>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
state_FL=biz_rank_pdf.loc[biz_rank_pdf['state'] == "FL"]   
bins = np.arange(0, 4820, 20)
labels = np.arange(240, 0, -1)
state_FL['Relevance'] = pd.cut(biz_rank_pdf['Rank'], bins=bins, labels=labels)

<command-1781364522751941>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
state_OR=biz_rank_pdf.loc[biz_rank_pdf['state'] == "OR"]   
bins = np.arange(0, 4620, 20)
labels = np.arange(230, 0, -1)
state_OR['Relevance'] = pd.cut(biz_rank_pdf['Rank'], bins=bins, labels=labels)

<command-1781364522751940>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
state_GA=biz_rank_pdf.loc[biz_rank_pdf['state'] == "GA"]   
bins = np.arange(0, 3800, 20)
labels = np.arange(189, 0, -1)
state_GA['Relevance'] = pd.cut(biz_rank_pdf['Rank'], bins=bins, labels=labels)

<command-1781364522751939>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
state_BC=biz_rank_pdf.loc[biz_rank_pdf['state'] == "BC"]   
bins = np.arange(0, 3780, 20)
labels = np.arange(188, 0, -1)
state_BC['Relevance'] = pd.cut(biz_rank_pdf['Rank'], bins=bins, labels=labels)

<command-1781364522751948>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
state_TX=biz_rank_pdf.loc[biz_rank_pdf['state'] == "TX"]   
bins = np.arange(0, 3480, 20)
labels = np.arange(173, 0, -1)
state_TX['Relevance'] = pd.cut(biz_rank_pdf['Rank'], bins=bins, labels=labels)

<command-1781364522751947>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
state_OH=biz_rank_pdf.loc[biz_rank_pdf['state'] == "OH"]   
bins = np.arange(0, 2580, 20)
labels = np.arange(128, 0, -1)
state_OH['Relevance'] = pd.cut(biz_rank_pdf['Rank'], bins=bins, labels=labels)

<command-1781364522751946>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
state_WA=biz_rank_pdf.loc[biz_rank_pdf['state'] == "WA"]   
bins = np.arange(0, 570, 10)
labels = np.arange(56, 0, -1)
state_WA['Relevance'] = pd.cut(biz_rank_pdf['Rank'], bins=bins, labels=labels)

<command-1781364522751945>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
state_CO=biz_rank_pdf.loc[biz_rank_pdf['state'] == "CO"]   
bins = np.arange(0, 540, 10)
labels = np.arange(53, 0, -1)
state_CO['Relevance'] = pd.cut(biz_rank_pdf['Rank'], bins=bins, labels=labels)

<command-1781364522751944>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
frames = [state_MA, state_FL, state_OR,state_GA,state_BC,state_TX,state_OH,state_WA,state_CO]
biz_rank_pdf = pd.concat(frames)

### 4.2 Train Data

In [0]:
le = preprocessing.LabelEncoder()
cat=le.fit_transform(biz_rank_pdf["state"])

In [0]:
biz_rank_pdf["state"]=cat

In [0]:
train=biz_rank_pdf.copy()

In [0]:
del train["business_perf"]
del train["city"]
del train["address"]
del train["categories"]
del train["latitude"]
del train["longitude"]
del train["postal_code"]
del train["tips_count"]
del train["review_count"]
del train["review_ave_stars"]
del train["business_id"]
del train["cluster"]
del train["name"]
del train["Rank"]

In [0]:
scaler = MinMaxScaler()
train[["review_ss","tips_ss",'Cust_Sat_Score']] = scaler.fit_transform(
    train[["review_ss","tips_ss",'Cust_Sat_Score']])

In [0]:
gss = GroupShuffleSplit(test_size=.2, n_splits=1, random_state = 7).split(train, groups=train['state'])

X_train_inds, X_test_inds = next(gss)

train_data= train.iloc[X_train_inds]
X_train = train_data.loc[:, ~train_data.columns.isin(['state','Relevance'])]
y_train = train_data.loc[:, train_data.columns.isin(['Relevance'])]

groups = train_data.groupby('state').size().to_frame('size')['size'].to_numpy()

test_data= train.iloc[X_test_inds]

#We need to keep the id for later predictions
X_test = test_data.loc[:, ~test_data.columns.isin(['Relevance'])]
y_test = test_data.loc[:, test_data.columns.isin(['Relevance',"state"])]

In [0]:
model = xgb.XGBRanker(  
    tree_method='hist',
    booster='gbtree',
    objective='rank:pairwise',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=110, 
    subsample=0.75,
    )

model.fit(X_train, y_train, group=groups, verbose=True)

Out[210]: XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=0.9, enable_categorical=False,
          eta=0.05, gamma=0, gpu_id=-1, importance_type=None,
          interaction_constraints='', learning_rate=0.1, max_delta_step=0,
          max_depth=6, min_child_weight=1, missing=nan,
          monotone_constraints='()', n_estimators=110, n_jobs=2,
          num_parallel_tree=1, predictor='auto', random_state=42, reg_alpha=0,
          reg_lambda=1, scale_pos_weight=None, subsample=0.75,
          tree_method='hist', validate_parameters=1, verbosity=None)

### 4.3 Test Data

In [0]:
X_test_2=X_test.loc[X_test['state'] == 2] 
y_test_2=y_test.loc[y_test['state'] == 2] 
del y_test_2["state"]
predict_2=model.predict(X_test_2.loc[:, ~X_test_2.columns.isin(['state'])])
y_test_2["Rank"]=predict_2
y_test_2=y_test_2.sort_values(by='Rank', ascending=False)

<command-1781364522751972>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
X_test_7=X_test.loc[X_test['state'] == 7] 
y_test_7=y_test.loc[y_test['state'] == 7] 
del y_test_7["state"]
predict_7=model.predict(X_test_7.loc[:, ~X_test_7.columns.isin(['state'])])
y_test_7["Rank"]=predict_7
y_test_7=y_test_7.sort_values(by='Rank', ascending=False)

<command-1781364522751971>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
train_ndcg_score=(ndcg_score([y_test_2["Relevance"]], [y_test_2["Rank"]])+ndcg_score([y_test_7["Relevance"]], [y_test_7["Rank"]]))/2
print("Average NDCG Score:",round(train_ndcg_score,3))

Average NDCG Score: 1.0


## 5. Apply on Yelp Dataset

In [0]:
overall_train=biz_rank_pdf.copy()

In [0]:
del overall_train["business_perf"]
del overall_train["city"]
del overall_train["address"]
del overall_train["categories"]
del overall_train["latitude"]
del overall_train["longitude"]
del overall_train["postal_code"]
del overall_train["tips_count"]
del overall_train["review_count"]
del overall_train["review_ave_stars"]
del overall_train["business_id"]
del overall_train["cluster"]
del overall_train["name"]
del overall_train["Rank"]

In [0]:
scaler = MinMaxScaler()
overall_train[["review_ss","tips_ss",'Cust_Sat_Score']] = scaler.fit_transform(
    overall_train[["review_ss","tips_ss",'Cust_Sat_Score']])

In [0]:
del overall_train["Relevance"]

In [0]:
predict_overall=model.predict(overall_train.loc[:, ~overall_train.columns.isin(['state'])])

In [0]:
biz_rank_pdf["Score"]=predict_overall

In [0]:
og_rank=biz_rank_pdf["Rank"]
del biz_rank_pdf["Rank"]

In [0]:
cat_list=[0,1,2,3,4,5,6,7,8]
rank_order=[]

In [0]:
for i in range(len(cat_list)):
    df_iloc=biz_rank_pdf.loc[biz_rank_pdf['state'] == i]   
    df_iloc=df_iloc.sort_values(by='Score', ascending=False)
    rank=np.arange(start=1, stop=len(df_iloc)+1, step=1)
    df_iloc["StateRank"]=rank
    rank_order.append(df_iloc)
    
    
biz_rank_pdf=pd.concat(rank_order)

In [0]:
state_cat=le.inverse_transform(biz_rank_pdf["state"])
biz_rank_pdf["state"]=state_cat

In [0]:
GA_rank= biz_rank_pdf[biz_rank_pdf.state=="GA"]

## 6. Visualize Results

In [0]:
import plotly.express as px
fig = px.scatter(GA_rank, x="Relevance", y="Score", color='StateRank')
fig.show()

In [0]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure(data=[go.Table(
    header=dict(values=("Rank","Business"),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[GA_rank.StateRank, GA_rank.name],
               fill_color='lavender',
               align='left'))
])

fig.show()